# VANI

**Initial notebook configuration**

In [1]:
%load_ext autoreload
# %load_ext snakeviz
# %load_ext jupyterflame

In [2]:
import os
import sys
print(sys.executable)
print(sys.version)
print(sys.version_info)
sys.path.append(os.path.dirname(os.getcwd()))

/g/g92/izzet/projects/iopp/vani-analysis-tool/vani-venv/bin/python
3.7.2 (default, Feb 26 2019, 08:59:10) 
[GCC 4.9.3]
sys.version_info(major=3, minor=7, micro=2, releaselevel='final', serial=0)


In [3]:
# Filter unnecessary warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

**Import libraries**

In [4]:
import numpy as np
from IPython.display import HTML
from vani.analyzer import Analyzer, ClusterOptions, ClusterType
from vani.utils.data_filtering import filter_non_io_traces, split_io_mpi_trace, split_read_write_metadata

**Initialize VANI Analyzer**

In [5]:
%autoreload 2

# Initialize analyzer
co = ClusterOptions(cluster_type=ClusterType.Local)
vn = Analyzer(n_workers=8, cluster_options=co, debug=True)

Dashboard link: http://127.0.0.1:8787/status


**Analyze logs**

In [6]:
%%time
%autoreload 2

# Read logs
hacc_ddf = vn._read_parquet(log_dir="/p/lustre1/iopp/parquet_app_logs/hacc/nodes-32/workflow-0")
hacc_ddf = filter_non_io_traces(hacc_ddf)
hacc_io_ddf, hacc_mpi_ddf, hacc_trace_ddf = split_io_mpi_trace(hacc_ddf)
hacc_io_ddf_read, hacc_io_ddf_write, hacc_io_ddf_metadata = split_read_write_metadata(hacc_io_ddf)

Logs read (0.08190640527755022)
CPU times: user 432 ms, sys: 38.8 ms, total: 471 ms
Wall time: 476 ms


In [7]:
hacc_io_ddf_write.head()

,index,rank,thread_id,cat,tstart,tend,func_id,level,arg_count,args_1,...,args_7,args_8,args_9,args_10,duration,tmid,filename,size,count,bandwidth
35,36,0,312784,0,18.552891,18.677382,write,0,3,/p/gpfs1/iopp/temp/hacc_dir/test-Part00000000-...,...,,,,,0.124491,18.615135,/p/gpfs1/iopp/temp/hacc_dir/test-Part00000000-...,67108864.0,1.0,5.390671e+08
38,39,0,312784,0,18.677414,18.720806,write,0,3,/p/gpfs1/iopp/temp/hacc_dir/test-Part00000000-...,...,,,,,0.043392,18.699110,/p/gpfs1/iopp/temp/hacc_dir/test-Part00000000-...,67108864.0,1.0,1.546566e+09
41,42,0,312784,0,18.720913,18.840509,write,0,3,/p/gpfs1/iopp/temp/hacc_dir/test-Part00000000-...,...,,,,,0.119596,18.780712,/p/gpfs1/iopp/temp/hacc_dir/test-Part00000000-...,67108864.0,1.0,5.611274e+08
44,45,0,312784,0,18.840620,18.879042,write,0,3,/p/gpfs1/iopp/temp/hacc_dir/test-Part00000000-...,...,,,,,0.038422,18.859831,/p/gpfs1/iopp/temp/hacc_dir/test-Part00000000-...,67108864.0,1.0,1.746643e+09
47,48,0,312784,0,18.879133,18.925989,write,0,3,/p/gpfs1/iopp/temp/hacc_dir/test-Part00000000-...,...,,,,,0.046856,18.902561,/p/gpfs1/iopp/temp/hacc_dir/test-Part00000000-...,67108864.0,1.0,1.432239e+09


In [8]:
hacc_index_ddf = hacc_io_ddf_write[hacc_io_ddf_write['index'] == 48]
hacc_index_ddf[['index', 'rank', 'thread_id', 'cat', 'func_id', 'size', 'duration', 'bandwidth']].compute()

,index,rank,thread_id,cat,func_id,size,duration,bandwidth
47,48,0,312784,0,write,67108864.0,0.046856,1.432239e+09
47,48,8,312784,0,write,67108864.0,0.516968,1.298125e+08
47,48,16,312784,0,write,67108864.0,0.379725,1.767304e+08
47,48,24,312784,0,write,67108864.0,0.248793,2.697381e+08
47,48,32,312784,0,write,67108864.0,0.325239,2.063370e+08
...,...,...,...,...,...,...,...,...
47,48,1240,312784,0,write,67108864.0,0.299989,2.237042e+08
47,48,1248,312784,0,write,67108864.0,0.325312,2.062907e+08
47,48,1256,312784,0,write,67108864.0,0.048288,1.389767e+09
47,48,1264,312784,0,write,67108864.0,0.017643,3.803716e+09


In [9]:
hacc_index_ddf.groupby('func_id').sum().compute()

,index,rank,thread_id,cat,tstart,tend,level,arg_count,duration,tmid,size,count,bandwidth
func_id,,,,,,,,,,,,,
write,7584,99976,49419872,0,2404.310303,2417.383789,0,474,13.073431,2410.847168,1.060320e+10,158.0,6.822703e+11


In [10]:
hacc_tend = hacc_io_ddf_write['tend'].max().compute()
hacc_tstart = hacc_io_ddf_write['tstart'].min().compute()
hacc_tend - hacc_tstart

22.203325

In [11]:
hacc_total_size = hacc_io_ddf_write['size'].sum().compute()
hacc_total_size

848256040960.0

In [12]:
(hacc_total_size/1024.0/1024.0/1024.0)/(hacc_tend - hacc_tstart)

35.58025612543045

In [13]:
hacc_bw_max = hacc_io_ddf_write['bandwidth'].max().compute()
hacc_bw_min = hacc_io_ddf_write['bandwidth'].min().compute()
hacc_bw_avg = hacc_io_ddf_write['bandwidth'].mean().compute()

In [14]:
hacc_bw_max/1024.0/1024.0/1024.0, hacc_bw_min/1024.0/1024.0/1024.0, hacc_bw_avg/1024.0/1024.0/1024.0

(10.013139801375095, 0.017386063134839615, 3.936443315686725)

In [6]:
# Read logs
cm1_ddf = vn._read_parquet(log_dir="/p/lustre1/iopp/parquet_app_logs/cm1/nodes-32/workflow-4")
cm1_ddf = filter_non_io_traces(cm1_ddf)
cm1_io_ddf, cm1_mpi_ddf, cm1_trace_ddf = split_io_mpi_trace(cm1_ddf)
cm1_io_ddf_read, cm1_io_ddf_write, cm1_io_ddf_metadata = split_read_write_metadata(cm1_io_ddf)

Logs read (0.44505688454955816)


In [22]:
# cm1_io_ddf_read[cm1_io_ddf_read['duration'] < 0.0001]['index'].count().compute()
# cm1_io_ddf_read[cm1_io_ddf_read['duration'] < 0.0001].head()
import pandas as pd
pd.set_option('display.max_colwidth', 1000)
file_name = cm1_io_ddf_read[cm1_io_ddf_read['index'] == 4]['args_1'].compute()
file_name

3    /p/gpfs1/iopp/temp/cm1r20.3.2846827/config_files/les_ConvBoundLayer/namelist.input
Name: args_1, dtype: object

In [25]:
cm1_io_ddf_read[cm1_io_ddf_read['args_1'] == '/p/gpfs1/iopp/temp/cm1r20.3.2846827/config_files/les_ConvBoundLayer/namelist.input']['duration'].describe().compute()

count    1281.000000
mean        0.000030
std         0.000144
min         0.000007
25%         0.000009
50%         0.000010
75%         0.000011
max         0.001981
Name: duration, dtype: float64

In [16]:
cm1_io_ddf_write.head()

,index,rank,thread_id,cat,tstart,tend,func_id,level,arg_count,args_1,...,args_7,args_8,args_9,args_10,duration,tmid,filename,size,count,bandwidth
11,12,0,322192,0,3.099647,3.099776,write,0,3,/p/gpfs1/iopp/temp/cm1r20.3.2846827/config_fil...,...,,,,,0.000129,3.099711,/p/gpfs1/iopp/temp/cm1r20.3.2846827/config_fil...,122854.0,1.0,9.524714e+08
24,25,0,322192,0,3.376984,3.377056,write,0,3,/p/gpfs1/iopp/temp/cm1r20.3.2846827/config_fil...,...,,,,,0.000072,3.377020,/p/gpfs1/iopp/temp/cm1r20.3.2846827/config_fil...,2055.0,1.0,2.854071e+07
37,38,0,322192,0,5.628629,5.628732,write,0,3,/p/gpfs1/iopp/temp/cm1r20.3.2846827/config_fil...,...,,,,,0.000103,5.628681,/p/gpfs1/iopp/temp/cm1r20.3.2846827/config_fil...,5530.0,1.0,5.369097e+07
43,44,0,322192,0,5.633736,5.633796,write,0,3,/p/gpfs1/iopp/temp/cm1r20.3.2846827/config_fil...,...,,,,,0.000060,5.633766,/p/gpfs1/iopp/temp/cm1r20.3.2846827/config_fil...,272.0,1.0,4.527185e+06
49,50,0,322192,0,5.635682,5.635730,write,0,3,/p/gpfs1/iopp/temp/cm1r20.3.2846827/config_fil...,...,,,,,0.000048,5.635706,/p/gpfs1/iopp/temp/cm1r20.3.2846827/config_fil...,20.0,1.0,4.152776e+05


In [17]:
cm1_index_ddf = cm1_io_ddf_write[cm1_io_ddf_write['index'] == 30]
cm1_index_ddf[['index', 'rank', 'thread_id', 'cat', 'func_id', 'size', 'duration', 'bandwidth']].compute()

,index,rank,thread_id,cat,func_id,size,duration,bandwidth


In [18]:
cm1_io_ddf_write.groupby('rank').sum().compute()

,index,thread_id,cat,tstart,tend,level,arg_count,duration,tmid,size,count,bandwidth
rank,,,,,,,,,,,,
0,17361919,742008176,0,791691.625,791692.1875,0,6909,0.579684,791691.875,1.230853e+09,2303.0,1.225185e+12


In [19]:
cm1_tend = cm1_io_ddf_write['tend'].max().compute()
cm1_tstart = cm1_io_ddf_write['tstart'].min().compute()
cm1_tend - cm1_tstart

665.2058954238892

In [20]:
cm1_total_size = cm1_io_ddf_write['size'].sum().compute()
cm1_total_size

1230852944.0

In [21]:
(cm1_total_size/1024.0/1024.0/1024.0)/(cm1_tend - cm1_tstart)

0.0017232576269466933

In [22]:
cm1_bw_max = cm1_io_ddf_write['bandwidth'].max().compute()
cm1_bw_min = cm1_io_ddf_write['bandwidth'].min().compute()
cm1_bw_avg = cm1_io_ddf_write['bandwidth'].mean().compute()

In [23]:
cm1_bw_max/1024.0/1024.0/1024.0, cm1_bw_min/1024.0/1024.0/1024.0, cm1_bw_avg/1024.0/1024.0/1024.0

(10.893617021276595, 0.0, 0.4954592155763255)

In [24]:
import pandas as pd
x = [0, 100, 200, 640, 800, 1400, 1600, 1280]
x2 = [0.00037471, 0.0004455, 0.00051434, 0.00068318, 0.00065801, 0.0002929, 0.00017687, 0.00037471]
pd.cut(x2, bins=10, labels=[1,2,3,4,5,6,7,8,9,10])

[4, 6, 7, 10, 10, 3, 1, 4]
Categories (10, int64): [1 < 2 < 3 < 4 ... 7 < 8 < 9 < 10]

In [25]:
print(cm1_io_ddf_write.head(50)['duration'].sum())
print(cm1_io_ddf_write.head(50)['rank'].nunique())

0.025527239
1


In [26]:
print(hacc_io_ddf_write.head(50)['duration'].sum())
print(hacc_io_ddf_write.head(50)['rank'].nunique())

9.641438
5


In [27]:
hacc_io_ddf_write['index'].count().compute()

12800

In [28]:
print('Max xfer size:', hacc_io_ddf_write['size'].max().compute()/1024.0/1024.0)
print('Avg xfer size:', hacc_io_ddf_write['size'].mean().compute()/1024.0/1024.0)

Max xfer size: 128.0
Avg xfer size: 63.2


In [29]:
import numpy as np
xfer_size_labes = ["0", "4KB", "16KB", "64KB", "256KB", "1MB", "4MB", "16MB", "64MB", "128MB"]
xfer_sizes = np.multiply([0, 4, 16, 64, 256, 1024, 4*1024, 16*1024, 64*1024, 128*1024], 1024.0)
xfer_sizes_len = len(xfer_sizes)
hacc_io_ddf_write['xfer_size'] = ''
for xfer_size_i, xfer_size_start in enumerate(xfer_sizes):
    if xfer_size_i == xfer_sizes_len - 1:
        xfer_size_cond = hacc_io_ddf_write['size'].ge(xfer_size_start)
        hacc_io_ddf_write['xfer_size'] = hacc_io_ddf_write['xfer_size'].mask(xfer_size_cond, xfer_size_labes[xfer_size_i])
    else:
        xfer_size_end = xfer_sizes[xfer_size_i + 1]
        xfer_size_cond = hacc_io_ddf_write['size'].ge(xfer_size_start) & hacc_io_ddf_write['size'].lt(xfer_size_end)
        hacc_io_ddf_write['xfer_size'] = hacc_io_ddf_write['xfer_size'].mask(xfer_size_cond, xfer_size_labes[xfer_size_i])
hacc_io_ddf_write.tail()

,index,rank,thread_id,cat,tstart,tend,func_id,level,arg_count,args_1,...,args_8,args_9,args_10,duration,tmid,filename,size,count,bandwidth,xfer_size
938,939,1279,312784,0,5.177905,5.442342,write,0,3,/p/gpfs1/iopp/temp/hacc_dir/test-Part00001279-...,...,,,,0.264437,5.310123,/p/gpfs1/iopp/temp/hacc_dir/test-Part00001279-...,67108864.0,1.0,2.537804e+08,64MB
941,942,1279,312784,0,5.442410,5.454422,write,0,3,/p/gpfs1/iopp/temp/hacc_dir/test-Part00001279-...,...,,,,0.012012,5.448416,/p/gpfs1/iopp/temp/hacc_dir/test-Part00001279-...,67108864.0,1.0,5.586816e+09,64MB
944,945,1279,312784,0,5.454483,5.554087,write,0,3,/p/gpfs1/iopp/temp/hacc_dir/test-Part00001279-...,...,,,,0.099604,5.504285,/p/gpfs1/iopp/temp/hacc_dir/test-Part00001279-...,134217728.0,1.0,1.347512e+09,128MB
947,948,1279,312784,0,5.554137,5.614391,write,0,3,/p/gpfs1/iopp/temp/hacc_dir/test-Part00001279-...,...,,,,0.060254,5.584264,/p/gpfs1/iopp/temp/hacc_dir/test-Part00001279-...,33554432.0,1.0,5.568822e+08,16MB
951,952,1279,312784,0,6.436961,6.443243,write,0,3,/p/gpfs1/iopp/temp/hacc_dir/test-Part00001279-...,...,,,,0.006282,6.440102,/p/gpfs1/iopp/temp/hacc_dir/test-Part00001279-...,25165824.0,1.0,4.006115e+09,16MB


In [30]:
hacc_io_ddf_write.groupby('xfer_size')['xfer_size'].count().compute()

xfer_size
128MB    1280
16MB     2560
64MB     8960
Name: xfer_size, dtype: int64

In [31]:
cm1_io_ddf_write['index'].count().compute()

2303

In [32]:
print('Max xfer size:', cm1_io_ddf_write['size'].max().compute()/1024.0/1024.0)
print('Avg xfer size:', cm1_io_ddf_write['size'].mean().compute()/1024.0/1024.0)

Max xfer size: 16.0
Avg xfer size: 0.5096972817914982


In [33]:
import numpy as np
xfer_size_labes = ["0", "4KB", "16KB", "64KB", "256KB", "1MB", "4MB", "16MB", "64MB", "128MB"]
xfer_sizes = np.multiply([0, 4, 16, 64, 256, 1024, 4*1024, 16*1024, 64*1024, 128*1024], 1024.0)
xfer_sizes_len = len(xfer_sizes)
cm1_io_ddf_write['xfer_size'] = ''
for xfer_size_i, xfer_size_start in enumerate(xfer_sizes):
    if xfer_size_i == xfer_sizes_len - 1:
        xfer_size_cond = cm1_io_ddf_write['size'].ge(xfer_size_start)
        cm1_io_ddf_write['xfer_size'] = cm1_io_ddf_write['xfer_size'].mask(xfer_size_cond, xfer_size_labes[xfer_size_i])
    else:
        xfer_size_end = xfer_sizes[xfer_size_i + 1]
        xfer_size_cond = cm1_io_ddf_write['size'].ge(xfer_size_start) & cm1_io_ddf_write['size'].lt(xfer_size_end)
        cm1_io_ddf_write['xfer_size'] = cm1_io_ddf_write['xfer_size'].mask(xfer_size_cond, xfer_size_labes[xfer_size_i])
cm1_io_ddf_write.tail()

,index,rank,thread_id,cat,tstart,tend,func_id,level,arg_count,args_1,...,args_8,args_9,args_10,duration,tmid,filename,size,count,bandwidth,xfer_size


In [34]:
cm1_io_ddf_write.groupby('xfer_size')['xfer_size'].count().compute()

xfer_size
0         397
16KB      722
16MB       55
256KB       7
4KB      1092
4MB        27
64KB        3
Name: xfer_size, dtype: int64

In [35]:
cm1_io_ddf_read.head()

,index,rank,thread_id,cat,tstart,tend,func_id,level,arg_count,args_1,...,args_7,args_8,args_9,args_10,duration,tmid,filename,size,count,bandwidth
3,4,0,322192,0,2.823390,2.823399,read,0,3,/p/gpfs1/iopp/temp/cm1r20.3.2846827/config_fil...,...,,,,,0.000009,2.823394,/p/gpfs1/iopp/temp/cm1r20.3.2846827/config_fil...,16777216.0,1.0,1.851809e+12
16,17,0,322192,0,3.102013,3.102023,read,0,3,/p/gpfs1/iopp/temp/cm1r20.3.2846827/config_fil...,...,,,,,0.000010,3.102018,/p/gpfs1/iopp/temp/cm1r20.3.2846827/config_fil...,16777216.0,1.0,1.716311e+12
29,30,0,322192,0,3.413794,3.415101,read,0,3,/p/gpfs1/iopp/temp/cm1r20.3.2846827/config_fil...,...,,,,,0.001307,3.414447,/p/gpfs1/iopp/temp/cm1r20.3.2846827/config_fil...,16777216.0,1.0,1.283633e+10
15032,15033,1,322192,0,2.820954,2.820963,read,0,3,/p/gpfs1/iopp/temp/cm1r20.3.2846827/config_fil...,...,,,,,0.000009,2.820958,/p/gpfs1/iopp/temp/cm1r20.3.2846827/config_fil...,16777216.0,1.0,1.851809e+12
15038,15039,2,322192,0,2.813843,2.813853,read,0,3,/p/gpfs1/iopp/temp/cm1r20.3.2846827/config_fil...,...,,,,,0.000010,2.813848,/p/gpfs1/iopp/temp/cm1r20.3.2846827/config_fil...,16777216.0,1.0,1.675446e+12


In [36]:
cm1_io_ddf_read[cm1_io_ddf_read['rank'] == 5].head()

/g/g92/izzet/projects/iopp/vani-analysis-tool/vani-venv/lib/python3.7/site-packages/dask/dataframe/core.py:7117: UserWarning: Insufficient elements for `head`. 5 elements requested, only 1 elements available. Try passing larger `npartitions` to `head`.
  f"Insufficient elements for `head`. {n} elements requested, only {len(r)} "


,index,rank,thread_id,cat,tstart,tend,func_id,level,arg_count,args_1,...,args_7,args_8,args_9,args_10,duration,tmid,filename,size,count,bandwidth
15056,15057,5,322192,0,2.801046,2.801055,read,0,3,/p/gpfs1/iopp/temp/cm1r20.3.2846827/config_fil...,...,,,,,0.000009,2.80105,/p/gpfs1/iopp/temp/cm1r20.3.2846827/config_fil...,16777216.0,1.0,1.851809e+12


In [37]:
hacc_io_ddf_write.groupby('rank')['size'].sum().compute()

rank
0       662700032.0
1       662700032.0
2       662700032.0
3       662700032.0
4       662700032.0
           ...     
1275    662700032.0
1276    662700032.0
1277    662700032.0
1278    662700032.0
1279    662700032.0
Name: size, Length: 1280, dtype: float64

In [38]:
cm1_io_ddf_read.groupby('rank')['size'].sum().compute()

rank
0       50331648.0
1       16777216.0
2       16777216.0
3       16777216.0
4       16777216.0
           ...    
1275    16777216.0
1276    16777216.0
1277    16777216.0
1278    16777216.0
1279    16777216.0
Name: size, Length: 1280, dtype: float64

In [45]:
hacc_io_ddf_metadata.head()

,index,rank,thread_id,cat,tstart,tend,func_id,level,arg_count,args_1,...,args_4,args_5,args_6,args_7,args_8,args_9,args_10,duration,tmid,filename
4,5,0,312784,0,4.298360,4.298391,open,1,3,/dev/shm/job2154496201-35619-OMPI_COLL_IBM-0-c...,...,,,,,,,,0.000031,4.298375,/dev/shm/job2154496201-35619-OMPI_COLL_IBM-0-c...
5,6,0,312784,0,4.298439,4.298456,open,1,2,/dev/shm/job2154496201-35619-OMPI_COLL_IBM-0-c...,...,,,,,,,,0.000017,4.298448,/dev/shm/job2154496201-35619-OMPI_COLL_IBM-0-c...
6,7,0,312784,0,4.298470,4.298541,ftruncate,1,2,/dev/shm/job2154496201-35619-OMPI_COLL_IBM-0-c...,...,,,,,,,,0.000070,4.298506,
7,8,0,312784,0,4.298553,4.298555,close,1,1,/dev/shm/job2154496201-35619-OMPI_COLL_IBM-0-c...,...,,,,,,,,0.000002,4.298553,/dev/shm/job2154496201-35619-OMPI_COLL_IBM-0-c...
8,9,0,312784,0,4.299555,4.299704,unlink,1,1,/dev/shm/job2154496201-35619-OMPI_COLL_IBM-0-c...,...,,,,,,,,0.000149,4.299629,
